In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

In [4]:
batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnistdata', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./mnistdata', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

In [16]:
train_loader.dataset,len(train_loader.dataset)

(Dataset MNIST
     Number of datapoints: 60000
     Split: train
     Root Location: ./mnistdata
     Transforms (if any): Compose(
                              ToTensor()
                              Normalize(mean=(0.1307,), std=(0.3081,))
                          )
     Target Transforms (if any): None, 60000)

In [5]:
w1, b1 = torch.randn(200, 784, requires_grad=True),\
         torch.zeros(1,200, requires_grad=True)
w2, b2 = torch.randn(200, 200, requires_grad=True),\
         torch.zeros(1,200, requires_grad=True)
w3, b3 = torch.randn(10, 200, requires_grad=True),\
         torch.zeros(1,10, requires_grad=True)

In [6]:
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)

tensor([[ 3.0639e-01,  5.1388e-02,  2.5816e-02,  ...,  9.3717e-02,
         -4.5858e-02, -1.3057e-01],
        [-1.3789e-01,  7.8814e-02, -1.5594e-01,  ...,  7.7531e-02,
          4.8861e-02,  8.2250e-02],
        [ 1.2696e-03, -4.2819e-02, -6.7098e-02,  ...,  1.4160e-01,
         -8.7542e-02, -9.6662e-02],
        ...,
        [-1.3552e-01,  1.1679e-01,  1.0412e-02,  ...,  9.5724e-02,
          2.2451e-02, -3.1229e-02],
        [-7.2284e-02, -1.9237e-02,  1.3583e-01,  ...,  3.4662e-03,
         -1.0937e-02, -7.5768e-02],
        [ 1.2680e-01, -9.1756e-02, -3.0125e-02,  ..., -2.8894e-04,
         -1.3931e-02, -5.1417e-02]], requires_grad=True)

In [9]:
def forward(x):
    x = x@w1.t() + b1
    x = F.relu(x)
    x = x@w2.t() + b2
    x = F.relu(x)
    x = x@w3.t() + b3
    x = F.relu(x)
    return x 

In [10]:
optimizer = optim.SGD([w1, b1, w2, b2, w3, b3], lr=learning_rate)
criteon = nn.CrossEntropyLoss()

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)

        logits = forward(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum() #计算和

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.775871
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.924767
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.546057

Test set: Average loss: 0.0018, Accuracy: 8959/10000 (89%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.405401
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.334161
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.209312

Test set: Average loss: 0.0014, Accuracy: 9202/10000 (92%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.296367
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.172390
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.301806

Test set: Average loss: 0.0012, Accuracy: 9318/10000 (93%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.275121
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.245940
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.226609

Test set: Average loss: 0.0011, Accuracy: 9378/10000 (93%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.200382
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.294513
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.249885

Test set: Average loss: 0.0010